In [63]:
import pandas as pd
import requests
import io
from datetime import datetime
import database as db
from pymongo import MongoClient

In [64]:
def monday_of_week(year, week):
    first_day_of_year = pd.to_datetime(f'{year}-01-01')
    first_monday = first_day_of_year - pd.Timedelta(days=first_day_of_year.dayofweek)
    monday_of_given_week = first_monday + pd.Timedelta(weeks=week-1)
    return monday_of_given_week

In [65]:
def find_report():
    url_list = ['https://visionet.franceagrimer.fr/Pages/OpenDocument.aspx?fileurl=SeriesChronologiques%2fproductions%20vegetales%2fgrandes%20cultures%2fetats%20des%20cultures%2fSCR-GRC-CEREOBS_M_depuis_2015-A24.xlsx&telechargersanscomptage=oui',
                'https://visionet.franceagrimer.fr/Pages/OpenDocument.aspx?fileurl=SeriesChronologiques%2fproductions%20vegetales%2fgrandes%20cultures%2fetats%20des%20cultures%2fSCR-GRC-CEREOBS_CP_depuis_2015-A25.xlsx&telechargersanscomptage=oui']
    dfList = []
    for url in url_list:
        try:
            r = requests.get(url)
            r.raise_for_status()
            tmpFr = pd.read_excel(io.BytesIO(r.content), sheet_name='Données France', skiprows=5)
            tmpRegion = pd.read_excel(io.BytesIO(r.content), sheet_name='Données régions', skiprows=5)
            tmpFr['Région'] = 'Moyenne France'
            dfList.append(tmpFr)
            dfList.append(tmpRegion)
        except requests.exceptions.RequestException as e:
            print(f"Error downloading the file: {e}")
    df = pd.concat(dfList)
    df = df.drop('Unnamed: 0', axis=1)
    df['Year'] = df['Semaine'].str.extract(r'(\d+)-S\d+')[0].astype(int)
    df['Week'] = df['Semaine'].str.extract(r'S(\d+)')[0].astype(int)
    df['Date'] = df.apply(lambda row: monday_of_week(row['Year'], row['Week']), axis=1) 
    df = df.sort_values(by='Date')
    return df

In [68]:
df = find_report()
df = df[['Culture', 'Région', 'Semaine', 'Semis', 'Levée', '6/8 feuilles visibles', 'Floraison femelle', 'Humidité du grain 50%', 'Récolte', 'Très mauvaises', 'Mauvaises', 'Assez bonnes', 'Bonnes', 'Très bonnes', 'Week', 'Year', 'Date', 'Début tallage', 'Épi 1 cm', '2 noeuds', 'Épiaison']]

In [70]:
dbname = db.get_database()
collection_name = dbname["dev_cond"]
last_doc = collection_name.find_one({})
if not last_doc:
    r = collection_name.insert_many(df.to_dict('records'))
print(r)